# Wrangling

In the following cells I perform the necessary steps in order to gather the data from multiple sources. There are three data sources that I will be collecting data from: 
 1. CSV that was handed to me
 2. Downloaded TSV file from online source
 3. JSON data from Twitter's API

In [1]:
#importing all of the ncessary libraries
import pandas as pd
import requests
import json
import numpy as np
import tweepy
import os

### 1. Downloading and Loading the CSV File into a Dataframe
The CSV file was downloaded from Udactiy and stored on my local machine in the same folder location as my Jupyter Notebook. The file was then loaded into a dataframe using pandas as described in the cells below.

In [2]:
#loading the archive file to a dataframe
df_archive = pd.read_csv('./twitter-archive-enhanced.csv')
df_archive.head() #verify the dataframe loads properly

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


### 2. Downloading the TSV File from the Internet
The next file needed to be downloaded programmatically from the internet using python. A URL was provided where I could download the file. Using the OS and Requests library in python, I was able to create a folder on my machine and make a request to the URL to download the file. 

After downloading the file, I was able to name the file based on the URL. Then, I stored the data onto a new dataframe using pandas.

In [3]:
#created the folder to store the file that I needed to download
folder_name = 'image_predictions'
#create the folder only if the foldername does not exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
#store the URL of the files location in url variable
url = ' https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
#store the request response in the response variable
response = requests.get(url)
#open the file path based on the foldername created in the first step
#create the file name based on the url, use all characters after the last '/', then write the file to the specified location
with open(os.path.join(folder_name,url.split('/')[-1]), mode = 'wb') as file:
    file.write(response.content)

In [4]:
#since the file is a tsv open the file using pandas read_csv but use sep as \t for tabs and create the dataframe
df_image_pred = pd.read_csv(folder_name + '/' + url.split('/')[-1], sep='\t')
df_image_pred.head() #verify the dataframe created loads properly

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### 3. Accessing JSON data from Twitter's API
The next data set would have to be accessed using Twitter's API. After following the instructions in the class, I was able to setup my own Twitter developer access and get my own authentication and token keys. I followed the instructions on how to access and get data from the Twitter API and used the Tweepy API class to get data. 

First, I needed to get the Tweet IDs from the Archive file (first file) that I loaded. After getting the Tweet IDs stored in a variable, I ran some simple tests to understand how the data was being returned using a single Tweet ID. I made sure I understood how to use the Tweepy API, get the JSON data, and download the JSON data for one Tweet ID before doing the entire list of tweet IDs. The rest of the steps I took are listed next to the cells below.

In [5]:
#twitter API variables for authentication and access
consumer_key = 'mKkUiGXd08ESNBEvItFbUSGhm'
consumer_secret = 'KrmjP09QpsvoLxhTi8rGqFVKRSQyIwTD3x3MJfB8S2MsuhhC0C'
access_token = '819966980228681728-fWy7RqGAkgkX7phMbQm4SYtJEXW2Jbv'
access_secret = 'lKp25U01GkeTGRHwtyllMHvYClTBxPJuUKsXqko1rSH6N'
#authentication based on twitter documentation
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
#use the tweepy API class to authenticate and also set wait on rate limit to True so that it waits when the limit is reached
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [6]:
#retweet count and favorite ("like") count at minimum,

In [7]:
#perform a test on one tweet ID to ensure that the connection is successful, use tweet_mode extended based on recommendation
tweet = api.get_status(892420643555336193, tweet_mode='extended')
#store the response as a json
tweet_json = tweet._json
#verify that I can read the json response as a dict using retweet_count as an example 
tweet_json['retweet_count']
#retweet_count
#favorite_count

8310

In [8]:
#perform a simple test of creating a new file and writing all of the contents to the file
#commenting the below code out since I don't need it anymore
'''
with open('tweet_json.txt', 'w') as outfile:  
    json.dump(tweet_json, outfile)
'''

"\nwith open('tweet_json.txt', 'w') as outfile:  \n    json.dump(tweet_json, outfile)\n"

#### 3a. Downloading all of the JSON data into a text file
I stored all of the Tweet IDs from the archive data frame into a list which I would use when sending requests to the Twitter API. I created a simple script to help me loop through all of the Tweet IDs and store each JSON data for each Tweet in its own line in the text file. 

Because the script took a very long time to run (around 30 minutes), and since the script stores the text file on my local machine after it runs I didn't really need to run it again, I have commented out the script below. 

In [9]:
tweet_id = df_archive['tweet_id'] #store all of the tweet IDs in df_archive into a list
tweet_errors = {} #creating a dictionary that will be used for the exceptions
''' 
### As stated in the above markdown cell, commenting out the rest of the download script as I like to rerun the entire the notebook and make it look clean ###

import time #importing the time library as suggessted to measure how long the script is running
start = time.time() #starting the timer
for tweetid in tweet_id: #starting the for loop for all Tweet IDs in the list created above
    try: #creating a try-except block to catch all of the exceptions as some tweets were deleted
        tweet = api.get_status(tweetid, tweet_mode='extended') #use the get_status from the Tweepy API to get the JSON data for each tweet ID
        tweet_json = tweet._json #convert the data into json--found the _json function online
        with open('tweet_json.txt', 'a') as outfile: #open a new file called tweet_json with the append action so that all of the data gets written to the file instead of overwriting it
            json.dump(tweet_json, outfile) #use the json.dump function to load the json data into the text file
            outfile.write("\n") #this creates a new line on the file for the next tweet
        end = time.time() #this stops the time for the last data that was appended to the file
        total_time = end-start #calculate the total time from start to finish of last load
        print(tweetid, (total_time/60)) #print out the tweet ID and the total time in minutes to see the progress
    except Exception as e: #for all exceptions store in this exception block
        print(str(tweetid) +": "+str(e)) #print the tweet ID along witht he exception error message
        tweet_errors[tweetid]=tweet_json #store the data in the tweet_errors dict that was created earlier
'''

' \n### As stated in the above markdown cell, commenting out the rest of the download script as I like to rerun the entire the notebook and make it look clean ###\n\nimport time #importing the time library as suggessted to measure how long the script is running\nstart = time.time() #starting the timer\nfor tweetid in tweet_id: #starting the for loop for all Tweet IDs in the list created above\n    try: #creating a try-except block to catch all of the exceptions as some tweets were deleted\n        tweet = api.get_status(tweetid, tweet_mode=\'extended\') #use the get_status from the Tweepy API to get the JSON data for each tweet ID\n        tweet_json = tweet._json #convert the data into json--found the _json function online\n        with open(\'tweet_json.txt\', \'a\') as outfile: #open a new file called tweet_json with the append action so that all of the data gets written to the file instead of overwriting it\n            json.dump(tweet_json, outfile) #use the json.dump function to 

#### 3b. Reading the text file line by line and loading the data into a dataframe
This next section took the longest time in the wrangling process. Multiple iterations were taken to get the data correctly and I had to look at the JSON data in the text file multiple times for various tweets to ensure that I was getting the data properly.

I created the below script to load the text file and then read each line and store the attributes that I wanted into a dataframe. The minimum required attributes were Tweet ID, Retweet Count, and Favorite Count. I had to look through the text file to get the actual attribute names in the JSON data using one Tweet ID as an example. 

I also decided that I wanted to pull Full Text, URL, and User Mentions just to see if I could. This would prove to be more difficult than I thought. The Full Text was easy to pull as it was just like the other minimum required fields. However, the URL and User Mentions were both nested in the JSON data tree. So, I had to go multiple layers to access both. 

I noticed that User Mentions were empty for a lot of data (more than 2,000) so I decided to remove that from the script. 

URL was nested for the majority of tweets in the same location. However, there were some exceptions that came up which is why I added the try-except blocks to the first part of the dataframe script. Some URLs were nested in different parts of the JSON data.

In [10]:
#create a variable for the file that was created from step 3a with encoding set to utf-8
input_file = open('tweet_json.txt','r',encoding='utf-8')
df_list = [] #create a list that the attributes will be appended to
df_errors={} #create a dict for the exceptions
#start with the input_file variable
with input_file as f:
#start the for loop to go through each line of the input_file
    for line in f:
        try: #start the try-except block because of the exceptions
            data = json.loads(line) #use the json.load command to load the line as a json in the data variable
            tweet_id = data['id'] #start getting the attributes that I'm interested in starting with Tweet ID, since the data is in JSON format I can use the same functions as I would for a dict
            retweet_count = data['retweet_count'] #get the attribute retweet_count
            favorite_count = data['favorite_count'] #get the attribute favorite_count
            full_text = data['full_text'] #get the attribute full_text
            url = data['entities']['media'][0]['url'] #get the attribute url
            #append all of the attributes to df_list so that I can create a dataframe from the list
            df_list.append({'tweet_id':tweet_id, #create a key, value for tweet_id
                            'retweet_count':retweet_count, #create a key, value for retweet_count
                            'favorite_count':favorite_count, #create a key, value for favorite_count
                            'full_text':full_text, #create a key, value for full_text
                            'url':url, #create a key, value for url
                           })
            #use pandas DataFrame to convert the list into a dataframe with the appropriate columns
            df_tweepy = pd.DataFrame(df_list, columns = ['tweet_id',
                                                        'retweet_count',
                                                        'favorite_count',
                                                        'full_text',
                                                        'url'])
        #start the exception block as I noticed that some URLs are not stored in the same location of some tweets
        except Exception as e:
            print(str(tweet_id)+': '+str(e)) #print the exception message with the tweet ID
            df_errors[tweet_id]=line #store the line with the tweet ID into the dictionary
   

886267009285017600: 'media'
886054160059072513: 'media'
885518971528720385: 'media'
884247878851493888: 'media'
881633300179243008: 'media'
879674319642796034: 'media'
879130579576475649: 'media'
878604707211726852: 'media'
878404777348136964: 'media'
878316110768087041: 'media'
876537666061221889: 'media'
875097192612077568: 'media'
874434818259525634: 'media'
873337748698140672: 'media'
871166179821445120: 'media'
871102520638267392: 'media'
870726314365509632: 'media'
868639477480148993: 'media'
866720684873056260: 'media'
866094527597207552: 'media'
863471782782697472: 'media'
863427515083354112: 'media'
860981674716409858: 'media'
860177593139703809: 'media'
858860390427611136: 'media'
857214891891077121: 'media'
857062103051644929: 'media'
856602993587888130: 'media'
856330835276025856: 'media'
856288084350160898: 'media'
855862651834028034: 'media'
855860136149123072: 'media'
855857698524602368: 'media'
855818117272018944: 'media'
855245323840757760: 'media'
855138241867124737: 

In [11]:
len(df_errors.keys()) #find the number of exceptions in the dictionary 

273

#### 3c. Creating a new Dataframe Ignoring Exceptions
Next, I wanted to create a new dataframe now that I have all of my exceptions. I wanted to get the list of all the tweet IDs that had the exception then run a modified version of the script I created in step 3b to avoid any exceptions. 

Getting the URLs for each of the exceptions proved to be challenging as the URL was nested in various places for these tweets. I didn't have enough time to go through each of the exceptions or write the code to handle all scenarios. So, if an exception existed, I just ignored the URL.

In [12]:
tweet_errors = list(df_errors.keys()) #conver the Tweet IDs in the exception dict to a list
tweet_errors #verify the Tweet IDs

[886267009285017600,
 886054160059072513,
 885518971528720385,
 884247878851493888,
 881633300179243008,
 879674319642796034,
 879130579576475649,
 878604707211726852,
 878404777348136964,
 878316110768087041,
 876537666061221889,
 875097192612077568,
 874434818259525634,
 873337748698140672,
 871166179821445120,
 871102520638267392,
 870726314365509632,
 868639477480148993,
 866720684873056260,
 866094527597207552,
 863471782782697472,
 863427515083354112,
 860981674716409858,
 860177593139703809,
 858860390427611136,
 857214891891077121,
 857062103051644929,
 856602993587888130,
 856330835276025856,
 856288084350160898,
 855862651834028034,
 855860136149123072,
 855857698524602368,
 855818117272018944,
 855245323840757760,
 855138241867124737,
 852936405516943360,
 850333567704068097,
 849668094696017920,
 848213670039564288,
 847978865427394560,
 847617282490613760,
 846505985330044928,
 846139713627017216,
 845098359547420673,
 843981021012017153,
 841320156043304961,
 840761248237

In [13]:
#clearing the list and datagrame since I am going to re-run a modified version of the script in the below cell
df_list = [] 
df_tweepy = []

In [14]:
input_file = open('tweet_json.txt','r',encoding='utf-8') #store the file in a variable input_file
with input_file as f: #start by loading the file
    for line in f: #start the for loop
        data = json.loads(line) #use the json.load command to load the line as a json in the data variable
        tweet_id = data['id'] #start getting the attributes that I'm interested in starting with Tweet ID, since the data is in JSON format I can use the same functions as I would for a dict
        retweet_count = data['retweet_count'] #get the attribute retweet_count
        favorite_count = data['favorite_count'] #get the attribute favorite_count
        full_text = data['full_text'] #get the attribute full_text
        if  tweet_id not in tweet_errors: #if the Tweet ID does not exist in the Tweet ID exception list
            url = data['entities']['media'][0]['url'] #then use the following keys and store the value in the URL variable
        else: #if the Tweet ID does exist in the Tweet ID Exception list
            url=None #then don't put anything in the URL attribute
        #append all of the attributes to df_list so that I can create a dataframe from the list
        df_list.append({'tweet_id':tweet_id, #create a key, value for tweet_id
                        'retweet_count':retweet_count, #create a key, value for retweet_count
                        'favorite_count':favorite_count, #create a key, value for favorite_count
                        'full_text':full_text, #create a key, value for full_text
                        'url':url, #create a key, value for url
                        })
        #use pandas DataFrame to convert the list into a dataframe with the appropriate columns
        df_tweepy = pd.DataFrame(df_list, columns = ['tweet_id',
                                                    'retweet_count',
                                                    'favorite_count',
                                                    'full_text',
                                                    'url'])

In [15]:
df_tweepy.count() #verify the counts, some URLs will be missing or null

tweet_id          2340
retweet_count     2340
favorite_count    2340
full_text         2340
url               2067
dtype: int64